In [1]:
import numpy as np
import cv2
import onnx
import onnxruntime as ort
import numpy as np
from PIL import Image
from pathlib import Path
import vai_q_onnx

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import CIFAR10

import onnx
import onnxruntime as ort
from onnxruntime.quantization import CalibrationDataReader, QuantType, QuantFormat, CalibrationMethod, quantize_static

import os, shutil
from time import perf_counter

In [2]:
def unpickle(file):
    import pickle
    with open(file,'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

datafile = r'./data/cifar-10-batches-py/test_batch'
metafile = r'./data/cifar-10-batches-py/batches.meta'

data_batch_1 = unpickle(datafile) 
metadata = unpickle(metafile)

images = data_batch_1['data']
labels = data_batch_1['labels']
images = np.reshape(images,(10000, 3, 32, 32))

dirname = 'images'
if not os.path.exists(dirname):
   os.mkdir(dirname)

In [3]:
quant_dir = './models/quant'
models = os.listdir(quant_dir)

cpu_time = []
model_name = models[0]

quantized_model_path = os.path.join(quant_dir, model_name)
model = onnx.load(quantized_model_path)

use_aie = True

providers = ['CPUExecutionProvider']
provider_options = [{}]

session = ort.InferenceSession(model.SerializeToString(), providers=providers,
                            provider_options=provider_options)

elapsed = 0
correct = 0
im = images
image_array = np.array(im).astype(np.float32)
image_array = image_array/255
input_data = image_array

ts = perf_counter()
for i in range(10):
    outputs = session.run(None, {'input': input_data})
elapsed += (perf_counter() - ts)/10


In [4]:
print(outputs[0].shape)
for i in range(len(outputs)):
    output_array = outputs[0][i]
    predicted_class = np.argmax(output_array)
    predicted_label = metadata['label_names'][predicted_class]
    label = metadata['label_names'][labels[i]]
    if predicted_class == labels[i]:
        correct += 1
cpu_time.append(elapsed)
print(f"Model: {model_name}, Accuracy: {correct/10000}, Time: {elapsed}")

(10000, 10)
Model: ResNet10_1111_4.U8S8.onnx, Accuracy: 0.0001, Time: 1.0705274799999998


In [5]:
quant_dir = './models/quant'
models = os.listdir(quant_dir)

cpu_time = []
for model_name in models:
    quantized_model_path = os.path.join(quant_dir, model_name)
    model = onnx.load(quantized_model_path)

    use_aie = True

    providers = ['CPUExecutionProvider']
    provider_options = [{}]

    session = ort.InferenceSession(model.SerializeToString(), providers=providers,
                                provider_options=provider_options)
    
    elapsed = 0
    correct = 0
    for i in range(10000): 
        im = images[i]
        image_array = np.array(im).astype(np.float32)
        image_array = image_array/255
        input_data = np.expand_dims(image_array, axis=0)

        ts = perf_counter()
        outputs = session.run(None, {'input': input_data})
        elapsed += (perf_counter() - ts)

        output_array = outputs[0]
        predicted_class = np.argmax(output_array)
        predicted_label = metadata['label_names'][predicted_class]
        label = metadata['label_names'][labels[i]]
        if predicted_class == labels[i]:
            correct += 1
    cpu_time.append(elapsed)
    print(f"Model: {model_name}, Accuracy: {correct/10000}, Time: {elapsed}")

Model: ResNet10_1111_4.U8S8.onnx, Accuracy: 0.0845, Time: 1.8717528999991373
Model: ResNet10_1111_6.U8S8.onnx, Accuracy: 0.0888, Time: 2.5674116999992407
Model: ResNet10_1111_8.U8S8.onnx, Accuracy: 0.1128, Time: 3.5455244999999564
Model: ResNet10_22_4.U8S8.onnx, Accuracy: 0.0922, Time: 1.6064468000003274
Model: ResNet10_22_6.U8S8.onnx, Accuracy: 0.1236, Time: 2.2081593000012134
Model: ResNet10_22_8.U8S8.onnx, Accuracy: 0.0865, Time: 2.752194799999671
Model: ResNet12_2111_4.U8S8.onnx, Accuracy: 0.092, Time: 2.093540099999501
Model: ResNet12_2111_6.U8S8.onnx, Accuracy: 0.0957, Time: 3.0900235000015357
Model: ResNet12_2111_8.U8S8.onnx, Accuracy: 0.0968, Time: 4.159379399999267
Model: ResNet14_2211_4.U8S8.onnx, Accuracy: 0.0825, Time: 2.361932400000967
Model: ResNet14_2211_6.U8S8.onnx, Accuracy: 0.1028, Time: 3.282070699998229
Model: ResNet14_2211_8.U8S8.onnx, Accuracy: 0.0509, Time: 4.51731920000131
Model: ResNet14_222_4.U8S8.onnx, Accuracy: 0.1118, Time: 2.251398700000834
Model: ResNet14

In [6]:

print("CPU Time: ", cpu_time)
cpu_time_np = np.array(cpu_time)
np.save("cpu_time.npy", cpu_time_np)

CPU Time:  [1.8717528999991373, 2.5674116999992407, 3.5455244999999564, 1.6064468000003274, 2.2081593000012134, 2.752194799999671, 2.093540099999501, 3.0900235000015357, 4.159379399999267, 2.361932400000967, 3.282070699998229, 4.51731920000131, 2.251398700000834, 3.2573209999992514, 4.216187999999931, 2.7227459999969597, 4.188958599999779, 5.644059400001339, 3.629270600000268, 5.209090699999479, 7.496912099998184, 6.384880399999474, 9.405165899999645, 13.68932740000011]
